# Xception Model

## Part-1 Data Preprocessing

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.3.1'

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')
validation_set = train_datagen.flow_from_directory(
        'dataset/validation_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 3013 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 1060 images belonging to 2 classes.


In [ ]:
x,y = test_set.next()
x.shape

## Part-2 Building the Model

In [6]:
model = tf.keras.models.Sequential()

In [7]:
model.add(tf.keras.applications.Xception(include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",))

In [8]:
model.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 1000)              22910480  
_________________________________________________________________
dense (Dense)                (None, 1)                 1001      
Total params: 22,911,481
Trainable params: 22,856,953
Non-trainable params: 54,528
_________________________________________________________________


## Part-3 Training the Model

In [10]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [11]:
model.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
95/95 [==============================] - 199s 2s/step - loss: 0.6643 - accuracy: 0.7424 - val_loss: 0.6739 - val_accuracy: 0.6377
Epoch 2/25
95/95 [==============================] - 181s 2s/step - loss: 0.6233 - accuracy: 0.8915 - val_loss: 0.6166 - val_accuracy: 0.8594
Epoch 3/25
95/95 [==============================] - 185s 2s/step - loss: 0.5784 - accuracy: 0.9154 - val_loss: 0.5196 - val_accuracy: 0.9670
Epoch 4/25
95/95 [==============================] - 184s 2s/step - loss: 0.5062 - accuracy: 0.9290 - val_loss: 0.4537 - val_accuracy: 0.9642
Epoch 5/25
95/95 [==============================] - 180s 2s/step - loss: 0.4428 - accuracy: 0.9545 - val_loss: 0.4000 - val_accuracy: 0.9858
Epoch 6/25
95/95 [==============================] - 193s 2s/step - loss: 0.4168 - accuracy: 0.9363 - val_loss: 0.3849 - val_accuracy: 0.9736
Epoch 7/25
95/95 [==============================] - 179s 2s/step - loss: 0.3739 - accuracy: 0.9665 - val_loss: 0.4207 - val_accuracy: 0.9321
Epoch 8/25
95

In [ ]:
import matplotlib.pyplot as plt
(history.history.keys())

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_set, verbose=2)
print('\nTest Accuracy: ', test_acc)

In [ ]:
import pandas as pd
import seaborn as sn
import tensorflow as tf

model = tf.keras.models.load_model("planet.h5")
filenames = test_set.filenames
nb_samples = len(test_set)
y_prob=[]
y_act=[]
test_set.reset()
for _ in range(nb_samples):
    X_test,Y_test = test_set.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)
    
predicted_class = [list(training_set.class_indices.keys())[i.argmax()] for i in y_prob]
actual_class = [list(training_set.class_indices.keys())[i.argmax()] for i in y_act]

out_df =pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns=['predicted_class','actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'],out_df['predicted_class'],rownames=['Actual'], colnames=['Predicted'])

sn.heatmap(confusion_matrix, cmap='Blues', annot=True,fmt='d')
plt.show()
print('test_accuracy: {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

## Part-4 Single Prediction

In [12]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/check_2.jpg', target_size = (299, 299, 3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'healthy'
else:
  prediction = 'unhealthy'

In [13]:
print(prediction)

unhealthy
